# Computing the Fréchet mean polytrope using Kleene stars

In [20]:
using TropicalFrechetMeans
using Polyhedra
using Clarabel, CDDLib
using Oscar

The following function serves to compute the breakpoints of a tropical line segments according to Theorem 5.11. in "Essentials of Tropical Combinatorics" by Joswig.

In [21]:
breakpoints_of_tropical_line(p::Vector{T}, q::Vector{T}) where {T<:TropicalSemiringElem} = breakpoints_of_tropical_line(QQ.(p), QQ.(q))

function breakpoints_of_tropical_line(p,q)
    r = q - p
    σ = sortperm(r)

    V = Vector{typeof(r)}(undef, length(r))

    for k in 1:length(r)
        V[k] = [q[σ[1:k]]..., r[σ[k]] .+ p[σ[k+1:end]]...][σ]
    end
    
    return V
end

breakpoints_of_tropical_line (generic function with 2 methods)

In [22]:
function trop_normalize(x)
    return x .- first(x)
end

function trop_normalize(x::Vector{T}) where {T<:TropicalSemiringElem}
    return x ./ first(x)
end

function indices(v)
    findfirst(==(1), v), findfirst(==(-1), v)
end

indices (generic function with 1 method)

In [39]:
sample = [[-3,0,0], [0,-6,0], [0,0,-12]]
n = length(sample |> first)

num_FM = tropical_frechet_mean(sample) |> trop_normalize
@show num_FM
P = tropical_frechet_set(sample)

num_FM = Rational{Int64}[0, 0, -1]


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
5-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, -1], 1//1)
 HalfSpace(Rational{BigInt}[-1, 1, 0], 1//1)
 HalfSpace(Rational{BigInt}[0, 1, -1], 1//1)
 HalfSpace(Rational{BigInt}[-1, 0, 1], -1//1)
 HalfSpace(Rational{BigInt}[0, -1, 1], -1//1)

The following tells us that the FM polytrope is a single point modulo lineality.

In [42]:
vrep(P)

V-representation CDDGeneratorMatrix{Rational{BigInt}, GMPRational}:
1-element iterator of Vector{Rational{BigInt}}:
 Rational{BigInt}[1, 1, 0],
1-element iterator of Line{Rational{BigInt}, Vector{Rational{BigInt}}}:
 Line(Rational{BigInt}[1, 1, 1])

In [25]:
H = hrep(P)

H-representation CDDInequalityMatrix{Rational{BigInt}, GMPRational}:
5-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, -1], 1//1)
 HalfSpace(Rational{BigInt}[-1, 1, 0], 1//1)
 HalfSpace(Rational{BigInt}[0, 1, -1], 1//1)
 HalfSpace(Rational{BigInt}[-1, 0, 1], -1//1)
 HalfSpace(Rational{BigInt}[0, -1, 1], -1//1)

From the facet description, we construct the matrix $C\in\mathbb{T}^{n\times n}$ that realizes the FM polytrope $\overline{P}$ as a weighted digraph polyhedron $$x_i - x_j \leq C_{ij}.$$

In [26]:
T = tropical_semiring()
C = identity_matrix(T, n)
for h in halfspaces(H)
    setindex!(C, h.β, indices(h.a)...)
end
Matrix(C)

3×3 Matrix{TropicalSemiringElem{typeof(min)}}:
 (0)   ∞     (1)
 (1)   (0)   (1)
 (-1)  (-1)  (0)

The following uses the tropical vertex description of $\overline{P} = \mathrm{tconv}(C^*)$ as the tropical column span of the Kleene star $$C^* = I_n \oplus C \oplus C^2 \oplus \dots \oplus C^{n-1}.$$

In [27]:
V = Set{Vector}()
A = C^n

for i in 1:size(A,1)
    for j in 1:size(A,1)
        if i != j
            for v in breakpoints_of_tropical_line(A[:,i], A[:,j])
                push!(V, trop_normalize(v))
            end
        end
    end
end
A

[ (0)    (0)   (1)]
[ (0)    (0)   (1)]
[(-1)   (-1)   (0)]

The procedure above recovers the pseudovertices as expected, which give the classical vertex description of $\overline{P}$. In this case, this is a single point.

In [28]:
V

Set{Vector} with 1 element:
  QQFieldElem[0, 0, -1]

## Phylogenetic trees

In the following, we apply the above computation of a classical vertex description to a FM polytrope with many vertices.
The corresponding dataset consists of phylogenetic trees, meaning the both dimension and number of facets is relatively high.
For this reason, even the double description method is infeasible, which means we turn to above Kleene star-based computation instead.

In [29]:
using JSON3
using DataFrames

# Function to read the JSON file and convert to a list of matrices
function read_and_convert_json(file_path::String)
    # Read the JSON file
    json_data = JSON3.read(file_path)
    
    # Extract elements from the nested arrays
    elements = [x[1] for x in json_data]
    elements = rationalize.(10000 * elements, tol=1e-2)
    
    # Convert elements into matrices
    num_elements = length(elements)
    matrices = []
    
    for i in 1:64:num_elements
        # Get the next 64 elements
        matrix_elements = elements[i:min(i+63, num_elements)]
        
        # Convert to an 8x8 matrix if there are 64 elements, otherwise create a smaller matrix
        matrix_size = length(matrix_elements)
        sqrt_size = Int(sqrt(matrix_size))
        push!(matrices, reshape(matrix_elements, sqrt_size, sqrt_size))
    end
    
    return matrices
end

"""
Take a matrix of pairwise distances between taxa and returns the cophenetic vector.
"""
function cophenetic_from_distance(pairwise)
    n = size(pairwise, 1)
    coph = [pairwise[i, j] for i in 1:n-1 for j in i+1:n]
    return coph
end

# Read and convert the JSON file
file_path = "all_matrices.json"
matrices = read_and_convert_json(file_path)
taxa = ["Tg", "Et", "Cp", "Ta", "Bb", "Tt", "Pv", "Pf"]

coph_vecs = [cophenetic_from_distance(mat) for mat in matrices]

268-element Vector{Vector{Rational{Int64}}}:
 [3784, 6626, 9906, 6521, 11778, 8750, 7661, 7601, 10881, 7496  …  4901, 15579, 12551, 11462, 12194, 9167, 8078, 14217, 13128, 1089]
 [3485, 8484, 9427, 8865, 25257, 9257, 10300, 8814, 9756, 9195  …  3645, 25010, 9010, 10054, 24449, 8449, 9492, 20033, 21076, 2163]
 [3570, 5592, 6890, 4591, 4849, 5136, 4872, 3905, 6062, 3763  …  6609, 6867, 7154, 6890, 2383, 4367, 4103, 4625, 4361, 274]
 [1696, 4297, 5665, 6408, 4431, 4218, 4248, 4473, 5841, 6584  …  9039, 7062, 6849, 6880, 5542, 6984, 7014, 5006, 5037, 557]
 [1864, 8173, 6779, 10056, 10843, 9010, 9319, 8433, 7039, 10315  …  11759, 12546, 10713, 11022, 15116, 13283, 13592, 12332, 12642, 2722]
 [3801, 11567, 5222, 5725, 5642, 8167, 7548, 12836, 6491, 6994  …  6129, 6046, 8571, 7952, 601, 7410, 6791, 7327, 6708, 4443]
 [751, 29706, 6877, 4376, 5857, 9999, 8966, 29899, 7070, 4569  …  6079, 7560, 11702, 10669, 3359, 8305, 7272, 9787, 8754, 5104]
 [4080, 8200, 9699, 9924, 11942, 9302, 9463, 7689, 

In [30]:
"""
Check if a distance matrix defines a phylogenetic tree
"""
function is_phylogenetic_tree(D)
    n = size(D, 1)
    
    # Check if the matrix is symmetric and non-negative
    for i in 1:n
        for j in i:n
            if D[i, j] != D[j, i] || D[i, j] < 0
                return false
            end
        end
    end

    # Check the four-point condition
    for i in 1:n-3
        for j in i+1:n-2
            for k in j+1:n-1
                for l in k+1:n
                    # Calculate distances
                    D_ij_kl = D[i, j] + D[k, l]
                    D_ik_jl = D[i, k] + D[j, l]
                    D_il_jk = D[i, l] + D[j, k]
                    
                    # Check the four-point condition
                    if !(D_ij_kl >= D_ik_jl && D_ij_kl >= D_il_jk) &&
                       !(D_ik_jl >= D_ij_kl && D_ik_jl >= D_il_jk) &&
                       !(D_il_jk >= D_ij_kl && D_il_jk >= D_ik_jl)
                        return false
                    end
                end
            end
        end
    end
    
    return true
end

is_phylogenetic_tree

In [38]:
@time phylo_frech = tropical_frechet_set(coph_vecs)

 10.259706 seconds (94.03 M allocations: 7.270 GiB, 7.17% gc time, 0.12% compilation time)


Polyhedron CDDLib.Polyhedron{Rational{BigInt}}:
300-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 0//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1192//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 866//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 824//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 677//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1023//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0], 1023//1)
 HalfSpace(Rationa

In [32]:
n = length(coph_vecs |> first)
H = hrep(phylo_frech)

H-representation CDDInequalityMatrix{Rational{BigInt}, GMPRational}:
300-element iterator of HalfSpace{Rational{BigInt}, Vector{Rational{BigInt}}}:
 HalfSpace(Rational{BigInt}[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1], 0//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1192//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 866//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 824//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 677//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1023//1)
 HalfSpace(Rational{BigInt}[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0], 1023//

In [33]:
T = tropical_semiring(min)
C = identity_matrix(T, n)
for h in halfspaces(H)
    setindex!(C, h.β, indices(h.a)...)
end
Matrix(C)

28×28 Matrix{TropicalSemiringElem{typeof(min)}}:
 (0)  ∞            ∞      ∞       …  ∞       ∞       ∞       ∞       (0)
 ∞    (0)          ∞      ∞          ∞       (1245)  ∞       ∞       ∞
 ∞    ∞            (0)    ∞          (1512)  (1650)  ∞       (1684)  ∞
 ∞    ∞            ∞      (0)        (2361)  (1885)  (1872)  (1397)  ∞
 ∞    ∞            ∞      ∞          (1192)  ∞       ∞       (2233)  ∞
 ∞    ∞            ∞      ∞       …  ∞       (2538)  (2330)  (2238)  ∞
 ∞    ∞            ∞      ∞          ∞       ∞       (1956)  (2331)  ∞
 ∞    ∞            ∞      ∞          ∞       ∞       ∞       ∞       ∞
 ∞    (37091//33)  ∞      ∞          (1612)  (1750)  (1667)  (1763)  ∞
 ∞    (56528//33)  (908)  ∞          ∞       (1964)  (1747)  (1476)  ∞
 ⋮                                ⋱                  ⋮               
 ∞    (51380//33)  ∞      ∞          (2289)  (2216)  (2203)  (1728)  ∞
 ∞    ∞            ∞      ∞       …  ∞       (2149)  (2275)  (2183)  ∞
 ∞    ∞            ∞      ∞

In [34]:
Matrix(C^n)

28×28 Matrix{TropicalSemiringElem{typeof(min)}}:
 (0)     (0)          (0)          (0)     …  (0)     (0)     (0)     (0)
 (1617)  (0)          (1617)       (1617)     (1245)  (1023)  (1023)  (1617)
 (2058)  (2058)       (0)          (2058)     (1650)  (2058)  (1684)  (2058)
 (2741)  (2741)       (1826)       (0)        (1885)  (1872)  (1397)  (2741)
 (2560)  (2560)       (2071)       (2560)     (2560)  (2560)  (2233)  (2560)
 (2572)  (2572)       (2572)       (2572)  …  (2538)  (2330)  (2238)  (2572)
 (2646)  (2646)       (2646)       (2646)     (2646)  (1956)  (2331)  (2646)
 (0)     (0)          (0)          (0)        (0)     (0)     (0)     (0)
 (2165)  (37091//33)  (997)        (2165)     (1750)  (1667)  (1763)  (2165)
 (2341)  (56528//33)  (908)        (2341)     (1964)  (1747)  (1476)  (2341)
 ⋮                                         ⋱          ⋮               
 (2494)  (51380//33)  (57881//33)  (2494)     (2216)  (2203)  (1728)  (2494)
 (2497)  (2497)       (2497)       (249

In [35]:
V = Set{Vector}()
A = C^n

for i in 1:size(A,1)
    for j in 1:size(A,1)
        if i != j
            for v in breakpoints_of_tropical_line(A[:,i], A[:,j])
                push!(V, trop_normalize(v))
            end
        end
    end
end
V

Set{Vector} with 9789 elements:
  QQFieldElem[0, -22078//33, -2099, -235, -1000, -131, -19439//33, -2497, -4102…
  QQFieldElem[0, 168, 1216, 1148, -277, 312, 1481, -933, 648, -1165  …  138, 11…
  QQFieldElem[0, -17786//33, -1143, -379, -1763, -7589//33, -1737, -74942//33, …
  QQFieldElem[0, 2297, 2421, 2230, 2257, 1841, 1894, 932, 66, 2326  …  1932, 0,…
  QQFieldElem[0, -541, 255, 348, -1954, -1377, -505, -470, 321, 121  …  -757, 3…
  QQFieldElem[0, 28247//33, 19865//33, 0, 63293//33, 0, 9206//33, 37718//33, 0,…
  QQFieldElem[0, -1174, -1588, -1768, 74, -75, -1664, -2231, -2572, -746  …  -1…
  QQFieldElem[0, 1903, -594, -594, -594, 2052, -594, 2507, -594, 36926//33  …  …
  QQFieldElem[0, 226, 122, -975, -2058, -1035, 357, -1738, 502, -2058  …  199, …
  QQFieldElem[0, 1339, 2588, 798, 1418, 2572, 2839//33, 0, 2296, 2254  …  0, 13…
  QQFieldElem[0, -453, 733, -79, 641, 704, 799, 216, -1476, -1476  …  -1476, -1…
  QQFieldElem[0, -836, 229, -79, 213, 42, 334, -628, -1456, -584  …  -620, -2

In [36]:
ratV = reduce(hcat, map(collect(V)) do v
    QQ.(v)[2:end]
end) |> transpose

9789×27 transpose(::Matrix{QQFieldElem}) with eltype QQFieldElem:
 -22078//33  -2099      -235   -1000      …  -2497       -2497      9
 168         1216       1148   -277          569         -54        392
 -17786//33  -1143      -379   -1763         -94544//33  -1622      -935
 2297        2421       2230   2257          2608        2450       2646
 -541        255        348    -1954         -1954       -1954      413
 28247//33   19865//33  0      63293//33  …  2174        2402       0
 -1174       -1588      -1768  74            -1575       -1693      -2197
 1903        -594       -594   -594          1955        1703       2367
 226         122        -975   -2058         265         255        -2058
 1339        2588       798    1418          2082        2728       0
 ⋮                                        ⋱              ⋮          
 330         -2560      86     -2560      …  242         -66        -85
 0           2191       2575   0             2308        2542       0
 -

In [37]:
FMP = convex_hull(ratV)

Polyhedron in ambient dimension 27